<a href="https://colab.research.google.com/github/smiliyo55/psoCNN_Brain_Tumor/blob/main/main1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install import_ipynb
!pip install torchsummary
import import_ipynb

In [ ]:
from psoCNN1 import psoCNN
from population1 import Population
import particle1
import numpy as np
import time
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import ipywidgets as widgets
from IPython.display import display
from torchsummary import summary

In [ ]:
# Define default values for parameters
default_dataset = "Brain Tumor MRI Dataset"
default_number_runs = 1
default_number_iterations = 2
default_population_size = 3
default_batch_size_pso = 16
default_batch_size_full_training = 16
default_epochs_pso = 1
default_epochs_full_training = 20
default_max_conv_output_channels = 32
default_max_fully_connected_neurons = 128
default_min_layer = 3
default_max_layer = 6
default_probability_convolution = 0.25
default_probability_pooling = 0.74
default_probability_fully_connected = 0.01
default_max_conv_kernel_size = 7
default_Cg = 0.5
default_dropout = 0.5

# Create widgets for each parameter with larger titles
#dataset_widget = widgets.Text(value=default_dataset, description='Dataset:', style={'description_width': 'initial'})
number_runs_widget = widgets.IntSlider(value=default_number_runs, min=1, max=10, description='Number of Runs:', style={'description_width': 'initial'})
number_iterations_widget = widgets.IntSlider(value=default_number_iterations, min=1, max=20, description='Number of Iterations:', style={'description_width': 'initial'})
population_size_widget = widgets.IntSlider(value=default_population_size, min=0, max=20, description='Population Size:', style={'description_width': 'initial'})
batch_size_pso_widget = widgets.IntSlider(value=default_batch_size_pso, min=1, max=32, description='Batch Size (PSO):', style={'description_width': 'initial'})
batch_size_full_training_widget = widgets.IntSlider(value=default_batch_size_full_training, min=1, max=32, description='Batch Size (Full Training):', style={'description_width': 'initial'})
epochs_pso_widget = widgets.IntSlider(value=default_epochs_pso, min=1, max=10, description='Epochs (PSO):', style={'description_width': 'initial'})
epochs_full_training_widget = widgets.IntSlider(value=default_epochs_full_training, min=10, max=50, description='Epochs (Full Training):', style={'description_width': 'initial'})
max_conv_output_channels_widget = widgets.IntSlider(value=default_max_conv_output_channels, min=10, max=512, description='Max Conv Output Channels:', style={'description_width': 'initial'})
max_fully_connected_neurons_widget = widgets.IntSlider(value=default_max_fully_connected_neurons, min=10, max=512, description='Max Fully Connected Neurons:', style={'description_width': 'initial'})
min_layer_widget = widgets.IntSlider(value=default_min_layer, min=1, max=10, description='Min Layers:', style={'description_width': 'initial'})
max_layer_widget = widgets.IntSlider(value=default_max_layer, min=3, max=50, description='Max Layers:', style={'description_width': 'initial'})
probability_convolution_widget = widgets.FloatSlider(value=default_probability_convolution, min=0, max=1, step=0.01, description='Probability Convolution:', style={'description_width': 'initial'})
probability_pooling_widget = widgets.FloatSlider(value=default_probability_pooling, min=0, max=1, step=0.01, description='Probability Pooling:', style={'description_width': 'initial'})
probability_fully_connected_widget = widgets.FloatSlider(value=default_probability_fully_connected, min=0, max=1, step=0.01, description='Probability Fully Connected:', style={'description_width': 'initial'})
max_conv_kernel_size_widget = widgets.IntSlider(value=default_max_conv_kernel_size, min=1, max=10, description='Max Conv Kernel Size:', style={'description_width': 'initial'})
Cg_widget = widgets.FloatSlider(value=default_Cg, min=0, max=1, step=0.1, description='Cg:', style={'description_width': 'initial'})
dropout_widget = widgets.FloatSlider(value=default_dropout, min=0, max=1, step=0.1, description='Dropout:', style={'description_width': 'initial'})

# Arrange widgets into a layout
parameter_widgets = [ number_runs_widget, number_iterations_widget, population_size_widget,
    batch_size_pso_widget, batch_size_full_training_widget, epochs_pso_widget, epochs_full_training_widget,
    max_conv_output_channels_widget, max_fully_connected_neurons_widget, min_layer_widget, max_layer_widget,
    probability_convolution_widget, probability_pooling_widget, probability_fully_connected_widget,
    max_conv_kernel_size_widget, Cg_widget, dropout_widget
]

# Create a grid layout with two columns
parameter_grid = widgets.GridBox(parameter_widgets, layout=widgets.Layout(grid_template_columns="repeat(2, 50%)"))

# Display the widgets with dataset name as title
display(widgets.VBox([widgets.Label(value=default_dataset, style={'font-weight': 'bold', 'font-size': '20px', 'margin': 'auto', 'text-align': 'center'}),
                      parameter_grid], layout=widgets.Layout(align_items='center')))
#display(widgets.VBox([widgets.Label(value=default_dataset, style={'font-weight': 'bold', 'font-size': '20px'}), parameter_grid]))

# Create a button widget
#run_button = widgets.Button(description="Run Algorithm")

# Define a function to run the algorithm with the specified parameters
def run_algorithm(button):
    dataset = default_dataset
    number_runs = number_runs_widget.value
    number_iterations = number_iterations_widget.value
    population_size = population_size_widget.value
    batch_size_pso = batch_size_pso_widget.value
    batch_size_full_training = batch_size_full_training_widget.value
    epochs_pso = epochs_pso_widget.value
    epochs_full_training = epochs_full_training_widget.value
    max_conv_output_channels = max_conv_output_channels_widget.value
    max_fully_connected_neurons = max_fully_connected_neurons_widget.value
    min_layer = min_layer_widget.value
    max_layer = max_layer_widget.value
    probability_convolution = probability_convolution_widget.value
    probability_pooling = probability_pooling_widget.value
    probability_fully_connected = probability_fully_connected_widget.value
    max_conv_kernel_size = max_conv_kernel_size_widget.value
    Cg = Cg_widget.value
    dropout = dropout_widget.value

    ########### Run the algorithm ######################
    results_path = "/content/drive/MyDrive/Colab Notebooks/psoCNN2/results/" + dataset + "/"

    if not os.path.exists(results_path):
            os.makedirs(results_path)

    all_gBest_metrics = np.zeros((number_runs, 2))
    runs_time = []
    all_gbest_par = []
    best_gBest_acc = 0

    for i in range(number_runs):
        print("Run number: " + str(i))
        start_time = time.time()

        pso = psoCNN(dataset=dataset, n_iter=number_iterations, pop_size=population_size,
                      batch_size=batch_size_pso, epochs=epochs_pso, min_layer=min_layer, max_layer=max_layer,
                      conv_prob=probability_convolution, pool_prob=probability_pooling,
                      fc_prob=probability_fully_connected, max_conv_kernel=max_conv_kernel_size,
                      max_out_ch=max_conv_output_channels, max_fc_neurons=max_fully_connected_neurons,
                      dropout_rate=dropout)

        pso.fit(Cg=Cg, dropout_rate=dropout)

        print(pso.gBest_acc)

        # Plot current gBest
        matplotlib.use('Agg')
        plt.plot(pso.gBest_acc)
        plt.xlabel("Iteration")
        plt.ylabel("gBest acc")
        plt.savefig(results_path + "gBest-iter-" + str(i) + ".png")
        plt.close()
        plt.show()

        print('gBest architecture: ')
        print(pso.gBest)

        np.save(results_path + "gBest_inter_" + str(i) + "_acc_history.npy", pso.gBest_acc)

        np.save(results_path + "gBest_iter_" + str(i) + "_test_acc_history.npy", pso.gBest_test_acc)

        end_time = time.time()

        running_time = end_time - start_time

        runs_time.append(running_time)

        # Fully train the gBest model found
        n_parameters, gBest_metrics = pso.fit_gBest(batch_size=batch_size_full_training, epochs=epochs_full_training, dropout_rate=dropout)
        all_gbest_par.append(n_parameters)

        # Visualize training history
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(gBest_metrics['train loss'], label='Training Loss')
        plt.plot(gBest_metrics['val loss'], label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Loss History')

        plt.subplot(1, 2, 2)
        plt.plot(gBest_metrics['train accuracy'], label='Training Accuracy')
        plt.plot(gBest_metrics['val accuracy'], label='Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.title('Accuracy History')

        plt.savefig(results_path + "gBest-iter-history-" + str(i) + ".png")
        plt.tight_layout()
        plt.show()

        if gBest_metrics['val accuracy'][-1] >= best_gBest_acc:
          best_gBest_acc = gBest_metrics['val accuracy'][-1]
          torch.save(pso.gBest.model, results_path + "best-gBest-model")

        all_gBest_metrics[i, 0] = gBest_metrics['val loss'][-1]
        all_gBest_metrics[i, 1] = gBest_metrics['val accuracy'][-1]

        m = running_time//60
        s = running_time%60
        print("This run took: " + str(m) + " min " + str(s) + " sec")

          # Compute mean accuracy of all runs
        all_gBest_mean_metrics = np.mean(all_gBest_metrics, axis=0)

        np.save(results_path + "/time_to_run.npy", runs_time)

        # Save all gBest metrics
        np.save(results_path + "/all_gBest_metrics.npy", all_gBest_metrics)

        # Save results in a text file
        output_str = "All gBest number of parameters: " + str(all_gbest_par) + "\n"
        output_str = output_str + "All gBest test accuracies: " + str(all_gBest_metrics[:,1]) + "\n"
        output_str = output_str + "All running times: " + str(runs_time) + "\n"
        output_str = output_str + "Mean loss of all runs: " + str(all_gBest_mean_metrics[0]) + "\n"
        output_str = output_str + "Mean accuracy of all runs: " + str(all_gBest_mean_metrics[1]) + "\n"

        print(output_str)

        with open(results_path + "/final_results.txt", "w") as f:
            try:
                print(output_str, file=f)
            except SyntaxError:
                print >> f, output_str

# Link the button to the function
#run_button.on_click(run_algorithm)

# Center the creation of the run button
centered_run_button_creation = widgets.VBox([widgets.Button(description="Run Algorithm", layout=widgets.Layout(width='auto'))], layout=widgets.Layout(justify_content='center'))

# Extract the run button from the VBox
run_button = centered_run_button_creation.children[0]

# Link the button to the function
run_button.on_click(run_algorithm)

# Center the run button horizontally at the bottom
centered_run_button = widgets.VBox([run_button], layout=widgets.Layout(justify_content='center'))

# Display the centered run button
display(centered_run_button)

# Display the button
#display(run_button)


In [ ]:
dataset = "Brain Tumor MRI Dataset"
number_runs = 1
number_iterations = 10
population_size = 20

batch_size_pso = 16
batch_size_full_training = 16

epochs_pso = 1
epochs_full_training = 20

max_conv_output_channels = 18
max_fully_connected_neurons = 150

min_layer = 3
max_layer = 10

# Probability of each layer type (should sum to 1)
probability_convolution = 0.4
probability_pooling = 0.8
probability_fully_connected = 0.0

max_conv_kernel_size = 8

Cg = 0.5
dropout = 0.5

########### Run the algorithm ######################
results_path = "/content/drive/MyDrive/Colab Notebooks/psoCNN2/results/" + dataset + "/"

if not os.path.exists(results_path):
        os.makedirs(results_path)

all_gBest_metrics = np.zeros((number_runs, 2))
runs_time = []
all_gbest_par = []
best_gBest_acc = 0

for i in range(number_runs):
    print("Run number: " + str(i))
    start_time = time.time()

    pso = psoCNN(dataset=dataset, n_iter=number_iterations, pop_size=population_size,
                  batch_size=batch_size_pso, epochs=epochs_pso, min_layer=min_layer, max_layer=max_layer,
                  conv_prob=probability_convolution, pool_prob=probability_pooling,
                  fc_prob=probability_fully_connected, max_conv_kernel=max_conv_kernel_size,
                  max_out_ch=max_conv_output_channels, max_fc_neurons=max_fully_connected_neurons,
                  dropout_rate=dropout)

    pso.fit(Cg=Cg, dropout_rate=dropout)

    print(pso.gBest_acc)

    # Plot current gBest
    matplotlib.use('Agg')
    plt.figure()
    plt.plot(pso.gBest_acc)
    plt.xlabel("Iteration")
    plt.ylabel("gBest acc")
    plt.savefig(results_path + "gBest-iter-" + str(i) + ".png")
    plt.close()
    plt.show()

    print('gBest architecture: ')
    print(pso.gBest)

    np.save(results_path + "gBest_inter_" + str(i) + "_acc_history.npy", pso.gBest_acc)

    np.save(results_path + "gBest_iter_" + str(i) + "_test_acc_history.npy", pso.gBest_test_acc)

    end_time = time.time()

    running_time = end_time - start_time

    runs_time.append(running_time)

    # Fully train the gBest model found
    n_parameters, gBest_metrics = pso.fit_gBest(batch_size=batch_size_full_training, epochs=epochs_full_training, dropout_rate=dropout)
    all_gbest_par.append(n_parameters)

    np.save(results_path + "gBest_" + str(i) + "_metrics.npy", gBest_metrics)

    # Visualize training history
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(gBest_metrics['train loss'], label='Training Loss')
    plt.plot(gBest_metrics['val loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss History')

    plt.subplot(1, 2, 2)
    plt.plot(gBest_metrics['train accuracy'], label='Training Accuracy')
    plt.plot(gBest_metrics['val accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy History')

    plt.savefig(results_path + "gBest-iter-history-" + str(i) + ".png")
    plt.tight_layout()
    plt.close()
    plt.show()

    if gBest_metrics['val accuracy'][-1] >= best_gBest_acc:
      best_gBest_acc = gBest_metrics['val accuracy'][-1]
      torch.save(pso.gBest.model, results_path + "best-gBest-model")

    torch.save(pso.gBest.model, results_path + "gBest-model-" + str(i))

    all_gBest_metrics[i, 0] = gBest_metrics['val loss'][-1]
    all_gBest_metrics[i, 1] = gBest_metrics['val accuracy'][-1]

    m = int(running_time/60.0)
    s = running_time%60.0
    print("This run took: " + str(m) + " min" + str(s) + " sec")
    #print("This run took: " + str(running_time) + " sec.")

      # Compute mean accuracy of all runs
    all_gBest_mean_metrics = np.mean(all_gBest_metrics, axis=0)

    np.save(results_path + "/time_to_run.npy", runs_time)

    # Save all gBest metrics
    np.save(results_path + "/all_gBest_metrics.npy", all_gBest_metrics)

    # Save results in a text file
    output_str = "All gBest number of parameters: " + str(all_gbest_par) + "\n"
    output_str = output_str + "All gBest test accuracies: " + str(all_gBest_metrics[:,1]) + "\n"
    output_str = output_str + "All running times: " + str(runs_time) + "\n"
    output_str = output_str + "Mean loss of all runs: " + str(all_gBest_mean_metrics[0]) + "\n"
    output_str = output_str + "Mean accuracy of all runs: " + str(all_gBest_mean_metrics[1]) + "\n"

    print(output_str)

    with open(results_path + "/final_results.txt", "w") as f:
        try:
            print(output_str, file=f)
        except SyntaxError:
            print >> f, output_str